In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
### Load the trained model, scaler pickle, onehot
model = load_model('model.h5')

###  load the encoders and scaler

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geography.pkl','rb') as file:
    onehot_encoder_geography = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [3]:
input_data ={
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' :1,
    'IsActiveMember': 1,
    'Exited':1

}

In [4]:
geo_encoded = onehot_encoder_geography.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geography.get_feature_names_out(['Geography']))
geo_encoded_df

c:\code\Gen AI Learning\ANN E2E Regression Project\ann_regression_venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,600,France,Male,40,3,60000,2,1,1,1


In [6]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,600,France,1,40,3,60000,2,1,1,1


In [7]:
# Concatinating OneHot encoding
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [8]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,1,1.0,0.0,0.0


In [9]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.51554201,  0.91578261,  0.11095929, -0.6961808 , -0.26510306,
         0.80697705,  0.64453717,  0.96995125,  1.97686   ,  0.996257  ,
        -0.57908232, -0.57273139]])

In [10]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


array([[81715.82]], dtype=float32)

In [13]:
prediction_probability = prediction[0][0]
prediction_probability

np.float32(81715.82)